In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
import glasbey

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
g_palette = glasbey.create_palette()
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)
blue_palette = sns.cubehelix_palette(as_cmap=True, rot=-.25, light=.7)
blue_root_palette = sns.cubehelix_palette(5, rot=-.25, light=.7)

In [4]:
from src.utils.tracklets import import_tracklets

roots = ["embryo007", "embryo008", "embryo014a", "embryo016", "embryo018"]
datapath = Path().cwd().parent / "data" / "interim" / "confocal" 
plotpath = datapath / "plots" / "cc_progression"
plotpath.mkdir(exist_ok=True)

spots, tracklets, metadata, tracklets_joined = import_tracklets(datapath, roots)

In [7]:
print(tracklets_joined.columns)
print(spots[roots[0]].columns)

Index(['start_time', 'end_time', 'start_frame', 'end_frame', 'length',
       'source_spot', 'sink_spot', 'mean_ap_position', 'source_ap_position',
       'sink_ap_position', 'initial_x', 'initial_y', 'final_x', 'final_y',
       'initial_x_um', 'initial_y_um', 'final_x_um', 'final_y_um', 'track_id',
       'mean_edge_distance', 'track_n_tracklets', 'cycle', 'embryo',
       'tracklet_id', 'parent_tracklet', 'n_children', 'e_parent_id'],
      dtype='object')
Index(['Unnamed: 0', 'ID', 'track_id', 'tracklet_id', 'distance_from_edge',
       'parent_id', 'daughter_id', 'roi', 'FRAME', 'POSITION_X', 'POSITION_Y',
       'POSITION_Z', 'ELLIPSE_MAJOR', 'ELLIPSE_MINOR', 'ELLIPSE_THETA',
       'ELLIPSE_Y0', 'ELLIPSE_X0', 'ELLIPSE_ASPECTRATIO', 'CIRCULARITY',
       'AREA', 'SHAPE_INDEX', 'MEDIAN_INTENSITY_CH1', 'time', 'um_from_edge',
       'um_x', 'um_y', 'ap_position', 'edge_position', 'track_n_tracklets',
       'cycle'],
      dtype='object')


## 1. Division as a marker of progression

In [14]:
for root in roots:
    fig, axes = plt.subplot_mosaic("AAAB", figsize=(7, 4), sharey=True)
    ax = axes["A"]
    data = tracklets_joined[tracklets_joined["embryo"] == root]
    data = data[data["track_n_tracklets"]==31]
    data = data[data["cycle"] > 10]
    data = data[data["sink_ap_position"] > 0.5]
    sns.scatterplot(data, y="start_frame", x="sink_ap_position", hue="cycle", ax=ax, palette=blue_palette)
    ax.invert_yaxis()
    ax.set_title(root)
    ax.set_xlabel("AP position")
    ax.set_ylabel("Start time (min)")
    ax = axes["B"]
    sns.kdeplot(data, y="start_frame", hue="cycle", ax=ax, palette=blue_palette, common_norm=False)
    plt.savefig(plotpath / f"{root}_division_start_timing.png", dpi=300)
    plt.close()

In [25]:
def jitter(values, jitter=0.5):
    n = len(values)
    return values + np.random.uniform(-jitter, jitter, n)

for cycle in [11, 12, 13]:
    fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    data = tracklets_joined[tracklets_joined["cycle"] == cycle]
    data = data[data["sink_ap_position"] > 0.5]
    data = data[data["track_n_tracklets"] == 31]
    sns.scatterplot(y=jitter(data["length"], 0.02), x=data["sink_ap_position"], hue=data["embryo"], ax=ax, palette=blue_root_palette, s=6)
    ax.set_title(f"Cycle {cycle}")
    ax.set_xlabel("AP position")
    ax.set_ylabel("Length (min)")
    plt.savefig(plotpath / f"cycle_{cycle}_length_vs_ap.png", dpi=300)
    plt.close()

## 2. Area and intensity as a marker of progression

In [33]:
ft_spots = {}

for root in roots:
    spot = spots[root]
    ft_spot = spot[spot["track_n_tracklets"] == 31].copy()
    
    rawfile = datapath / root / f"{root}_MaxIP_bgs.tif"
    raw = tifffile.imread(rawfile)
    shape = raw.shape
    
    ft_spot["intensity_mean"] = np.nan
    ft_spot["normed_intensity_mean"] = np.nan
    ft_spot["normed_area"] = np.nan
    
    for cycle in [11, 12, 13]:
        ft_cycle = ft_spot[ft_spot["cycle"] == cycle].copy()
        print(f"Root: {root}, Cycle: {cycle}, Number of spots: {ft_cycle.shape[0]}") 
        
        for idx, spot in ft_cycle.iterrows():
            x, y = spot["POSITION_X"], spot["POSITION_Y"]
            t = round(spot["FRAME"])
            new_track_id = spot["track_id"]
            
            roi = [float(pt.lstrip("[ ").rstrip("] ")) for pt in spot["roi"].split(",")]
        
            xs = [round(pt + x) for pt in roi[::2]]
            ys = [round(pt + y) for pt in roi[1::2]]
        
            rr, cc = polygon(ys, xs, shape[1:])
            intensity_vals = raw[tuple([t] + [rr, cc])]
            
            ft_cycle.loc[idx, "intensity_mean"] = intensity_vals.mean()
       
        x = ft_cycle["intensity_mean"]
        ft_cycle["normed_intensity_mean"] = (x - x.mean()) / x.std()
        x = ft_cycle["AREA"]
        ft_cycle["normed_area"] = (x - x.mean()) / x.std()
        ft_spot[ft_spot["cycle"] == cycle] = ft_cycle
       
    ft_spots[root] = ft_spot     

Root: embryo007, Cycle: 11, Number of spots: 3573
Root: embryo007, Cycle: 12, Number of spots: 7332
Root: embryo007, Cycle: 13, Number of spots: 21365
Root: embryo008, Cycle: 11, Number of spots: 3002
Root: embryo008, Cycle: 12, Number of spots: 7047
Root: embryo008, Cycle: 13, Number of spots: 20462
Root: embryo014a, Cycle: 11, Number of spots: 4227
Root: embryo014a, Cycle: 12, Number of spots: 9963
Root: embryo014a, Cycle: 13, Number of spots: 28724
Root: embryo016, Cycle: 11, Number of spots: 6086
Root: embryo016, Cycle: 12, Number of spots: 14651
Root: embryo016, Cycle: 13, Number of spots: 45512
Root: embryo018, Cycle: 11, Number of spots: 7347
Root: embryo018, Cycle: 12, Number of spots: 17621
Root: embryo018, Cycle: 13, Number of spots: 55265


In [67]:
print(ft_spots[roots[0]].columns)
print(metadata[roots[0]])

Index(['Unnamed: 0', 'ID', 'track_id', 'tracklet_id', 'distance_from_edge',
       'parent_id', 'daughter_id', 'roi', 'FRAME', 'POSITION_X', 'POSITION_Y',
       'POSITION_Z', 'ELLIPSE_MAJOR', 'ELLIPSE_MINOR', 'ELLIPSE_THETA',
       'ELLIPSE_Y0', 'ELLIPSE_X0', 'ELLIPSE_ASPECTRATIO', 'CIRCULARITY',
       'AREA', 'SHAPE_INDEX', 'MEDIAN_INTENSITY_CH1', 'time', 'um_from_edge',
       'um_x', 'um_y', 'ap_position', 'edge_position', 'track_n_tracklets',
       'cycle', 'intensity_mean', 'normed_intensity_mean', 'normed_area'],
      dtype='object')
{'name': 'embryo007', 'frames_per_minute': 4, 'pixels_per_um': 3, 'a_x': -20, 'a_y': 240, 'p_x': 1390, 'p_y': 806, 'h': 1360, 'w': 1360, 'n_divisions': 4, 'division_times': [0, 7, 53, 105, 182]}


In [47]:
def moving_avg(data, window=5):
    return data.rolling(window=window, center=True).mean()

def rate_of_change(data, window=5):
    return data.rolling(window=window, center=True).mean().diff()

In [89]:
from collections import defaultdict
root = roots[1]
plot = False
t_data = defaultdict(list)

for root in roots:
    for cycle in [11, 12, 13]:
        fpm = metadata[root]["frames_per_minute"]
        data = ft_spots[root].copy()
        data = data[data["cycle"] == cycle]
        data["mv_area"] = np.nan
        data["mv_intensity"] = np.nan
        data["roc_area"] = np.nan
        data["roc_intensity"] = np.nan
        
        
        for tracklet in data["tracklet_id"].unique():
            if tracklet == 0:
                continue
            tracklet_data = data[data["tracklet_id"] == tracklet].copy()
            tracklet_data["mv_area"] = moving_avg(tracklet_data["normed_area"])
            tracklet_data["mv_intensity"] = moving_avg(tracklet_data["normed_intensity_mean"])
            tracklet_data["roc_area"] = rate_of_change(tracklet_data["normed_area"]) * fpm
            tracklet_data["roc_intensity"] = rate_of_change(tracklet_data["normed_intensity_mean"]) * fpm
            tracklet_data["dv"] = np.sqrt(tracklet_data["roc_area"]**2 + tracklet_data["roc_intensity"]**2)
            halfway = np.median(tracklet_data["time"])
            t_data["early_peak_time"].append(tracklet_data.loc[tracklet_data[tracklet_data["time"] < halfway]["dv"].idxmax(), "time"])
            t_data["late_peak_time"].append(tracklet_data.loc[tracklet_data[tracklet_data["time"] > halfway]["dv"].idxmax(), "time"])
            t_data["early_peak_ap_position"].append(tracklet_data.loc[tracklet_data[tracklet_data["time"] < halfway]["dv"].idxmax(), "ap_position"])
            t_data["late_peak_ap_position"].append(tracklet_data.loc[tracklet_data[tracklet_data["time"] > halfway]["dv"].idxmax(), "ap_position"])
            t_data["length"].append(tracklet_data["time"].max() - tracklet_data["time"].min())
            t_data["early_peak"].append(tracklet_data[tracklet_data["time"] < halfway]["dv"].max())
            t_data["late_peak"].append(tracklet_data[tracklet_data["time"] > halfway]["dv"].max())
            t_data["cycle"].append(cycle)
            t_data["root"].append(root)

            
            data.loc[tracklet_data.index] = tracklet_data
            
        data["dv"] = np.sqrt(data["roc_area"]**2 + data["roc_intensity"]**2)
        if plot:
            fig, ax = plt.subplots()
            sns.scatterplot(data, x="mv_area", y="mv_intensity", hue="dv", palette='gist_rainbow', s=(10 - (cycle - 10)*2))
            plt.title("Progression speed")
            plt.ylabel("Intensity Z-Score")
            plt.xlabel("Area Z-score")
            plt.savefig(plotpath / f"dv/{root}_{cycle}_dv.png", dpi=300)
            plt.close()
            fig, ax = plt.subplots()
            sns.lineplot(data=data[data["ap_position"]>0.5], x="FRAME", y="dv", errorbar="sd")
            plt.title("Progression speed")
            plt.ylabel("dv")
            plt.xlabel("Time")
            plt.savefig(plotpath / f"dv_over_time/{root}_{cycle}_dv_time2.png", dpi=300)
            plt.close()
            
t_data = pd.DataFrame(t_data)

In [90]:
t_data["int_length"] = t_data["late_peak_time"] - t_data["early_peak_time"]
t_data["remainder"] = t_data["length"] - t_data["int_length"]
t_data.head()


,early_peak_time,late_peak_time,early_peak_ap_position,late_peak_ap_position,length,early_peak,late_peak,cycle,root,int_length,remainder
0,5.75,10.00,0.078267,0.096153,11.0,1.367411,1.919280,11,embryo007,4.25,6.75
1,6.00,10.00,0.099984,0.107525,11.0,0.961500,1.929012,11,embryo007,4.00,7.00
2,2.25,10.75,0.647611,0.667152,11.5,0.844270,2.008479,11,embryo007,8.50,3.00
3,2.25,10.50,0.629888,0.639721,11.5,0.698279,2.283815,11,embryo007,8.25,3.25
4,5.50,10.50,0.525136,0.530901,11.5,0.700453,1.984378,11,embryo007,5.00,6.50


In [93]:
for root in roots:
    for cycle in [11, 12, 13]:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        ax = axes[0]
        data = t_data[t_data["root"] == root]
        data = data[data["cycle"] == cycle]
        data = data[data["early_peak_ap_position"] > 0.5]
        sns.scatterplot(data, x="early_peak_ap_position", y="int_length", hue="cycle", palette=blue_palette, ax=ax)
        ax.set_title("Interphase length")
        ax.set_xlabel("AP position")
        ax.set_ylabel("Interphase length (min)")
        
        ax = axes[1]
        sns.scatterplot(data, x="early_peak_ap_position", y="remainder", hue="cycle", palette=blue_palette, ax=ax)
        ax.set_title("Remaining length")
        ax.set_xlabel("AP position")
        ax.set_ylabel("remaining length (min)")
        plt.savefig(plotpath / f"peak_timing/{root}_{cycle}_interphase_length.png", dpi=300)
        plt.close()
